In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Label Class Limiting

In [ ]:
import os
import cv2 as cv
import numpy as np
from tqdm import tqdm

In [ ]:
classes = [[24],[23],[26,27,28],[11,12],[7,8],[32,33],[6,22],[21]]
          # 1    2       3         4      5      6      7     8  

def modify(label_image):
    img = label_image.copy()
    for i,r in enumerate(img):
        for j,c in enumerate(r):
            mod_flag = 0
            for idx,clas in enumerate(classes):
                if c in clas:
                    img[i,j] = idx+1
                    mod_flag = 1
            if mod_flag==0:
                img[i,j] = 0
    return img

In [ ]:
label_path = os.path.join('drive','MyDrive','CITY','Resized','labels')
file_order_path = os.path.join('drive','MyDrive','CITY','File Order')
label_path_des = os.path.join('drive','MyDrive','CITY','Resized','labels_limited')

for cat in ['train','val']: 
    
  lab_order_path = os.path.join(file_order_path,'label_{}_file_order.npy'.format(cat))
  lab_order = np.load(lab_order_path)

  for lab in tqdm(lab_order,total = len(lab_order)):
      
      labe_path = os.path.join(label_path,lab)
      label = cv.imread(labe_path,cv.IMREAD_UNCHANGED)
      label = modify(label)
      lab_des = os.path.join(label_path_des,lab)
      cv.imwrite(lab_des,label)

100%|██████████| 500/500 [1:30:38<00:00, 10.88s/it]


Model Building

In [ ]:
import tensorflow as tf
import os
import numpy as np
import cv2 as cv

In [ ]:
model = tf.keras.models.load_model( os.path.join('drive','MyDrive','CITY','fast_scnn.h5') )
model.evaluate(val_dataset)

14/84 [====>.........................] - ETA: 16:49 - loss: 1.2069 - accuracy: 0.5794

In [ ]:
def conv_block(inputs,conv_type,kernel,kernel_size,strides,padding='same',relu=True):
    if conv_type == 'ds':
        x = tf.keras.layers.SeparableConv2D(kernel,kernel_size,padding=padding,strides=strides)(inputs)
    else:
        x = tf.keras.layers.Conv2D(kernel,kernel_size,padding=padding,strides=strides)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    if relu:
        x = tf.keras.activations.relu(x)
    return x

def _res_bottleneck(inputs,filters,kernel,t,s,r=False):
  tchannel = tf.keras.backend.int_shape(inputs)[-1]*t
  x = conv_block(inputs,'conv',tchannel,(1,1),strides=(1,1))
  x = tf.keras.layers.DepthwiseConv2D(kernel,strides=(s,s),depth_multiplier=1,padding='same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.activations.relu(x)
  x = conv_block(x,'conv',filters,(1,1),strides=(1,1),padding='same',relu=False)
  if r:
      x = tf.keras.layers.add([x,inputs])
  return x

def bottleneck_block(inputs,filters,kernel,t,strides,n):
    x = _res_bottleneck(inputs,filters,kernel,t,strides)
    for i in range(1,n):
        x = _res_bottleneck(x,filters,kernel,t,1,True)
    return x

def pyramid_pooling_block(input_tensor,bin_sizes):
    concat_list = [input_tensor]
#     w = 64
#     h = 32
    w = 32
    h = 64
    for bin_size in bin_sizes:
        x = tf.keras.layers.AveragePooling2D(pool_size=(w//bin_size,h//bin_size),strides=(w//bin_size,h//bin_size))(input_tensor)
        x = tf.keras.layers.Conv2D(128,3,2,padding='same')(x)
        x = tf.keras.layers.Lambda(lambda x:tf.image.resize(x,(w,h)))(x)
        concat_list.append(x)
    return tf.keras.layers.concatenate(concat_list)

In [ ]:
input_layer = tf.keras.layers.Input(shape=(512,1024,3),name='input_layer') #(2048,1024,3)
# input_layer = tf.keras.layers.BatchNormalization()(input_layer)
lds_layer = conv_block(input_layer,'conv',32,(3,3),strides=(2,2))
lds_layer = conv_block(lds_layer,'ds',48,(3,3),strides=(2,2))
lds_layer = conv_block(lds_layer,'ds',64,(3,3),strides=(1,1)) 
gfe_layer = bottleneck_block(lds_layer,64,(3,3),t=6,strides=2,n=3) 
gfe_layer = bottleneck_block(gfe_layer,96,(3,3),t=6,strides=2,n=3)
gfe_layer = bottleneck_block(gfe_layer,128,(3,3),t=6,strides=1,n=3) 
gfe_layer = pyramid_pooling_block(gfe_layer,[2,4,6,8]) 
ff_layer1 = conv_block(lds_layer,'conv',128,(1,1),padding='same',strides=(1,1),relu=False)
ff_layer2 = tf.keras.layers.UpSampling2D((4,4))(gfe_layer)
ff_layer2 = tf.keras.layers.DepthwiseConv2D(128,strides=(1,1),depth_multiplier=1,padding='same')(ff_layer2)
ff_layer2 = tf.keras.layers.BatchNormalization()(ff_layer2)
ff_layer2 = tf.keras.activations.relu(ff_layer2)
ff_layer2 = tf.keras.layers.Conv2D(128,1,1,padding='same',activation=None)(ff_layer2)
ff_final = tf.keras.layers.add([ff_layer1,ff_layer2])
ff_final = tf.keras.layers.BatchNormalization()(ff_final)
ff_final = tf.keras.activations.relu(ff_final)
classifier = tf.keras.layers.SeparableConv2D(128,(3,3),padding='same',strides=(1,1),name="DSConv1_classifier")(ff_final)
classifier = tf.keras.layers.BatchNormalization()(classifier)
classifier = tf.keras.activations.relu(classifier)

classifier = tf.keras.layers.SeparableConv2D(128,(3,3),padding='same',strides=(1,1),name="DSConv2_classifier")(classifier)
classifier = tf.keras.layers.BatchNormalization()(classifier)
classifier = tf.keras.activations.relu(classifier)

classifier = conv_block(classifier,'conv',9,(1,1),strides=(1,1),padding='same',relu=True) # classes number

classifier = tf.keras.layers.Dropout(0.1)(classifier)

classifier = tf.keras.layers.UpSampling2D((4,4))(classifier) #(8,8)
classifier = tf.keras.activations.softmax(classifier)

# compiling model
fast_scnn = tf.keras.Model(inputs=input_layer, outputs=classifier, name = 'Fast_SCNN')
optimizer = tf.keras.optimizers.SGD(momentum=0.9,lr=0.12) 
weights = os.path.join('drive','MyDrive','CITY','fast_scnn_weights_2.hdf5')
fast_scnn.load_weights(weights)
fast_scnn.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
image_path_og = os.path.join('drive','MyDrive','CITY','Resized','images')
label_path_og = os.path.join('drive','MyDrive','CITY','Resized','labels_limited')
file_order_path = os.path.join('drive','MyDrive','CITY','File Order')
  
def load_data(cat):
  x = []
  y = []
  
  img_order_path = os.path.join(file_order_path,'img_{}_file_order.npy'.format(cat))
  lab_order_path = os.path.join(file_order_path,'label_{}_file_order.npy'.format(cat))
  img_order = np.load(img_order_path)
  lab_order = np.load(lab_order_path)
  
  for img,lab in zip(img_order,lab_order):
    image_path = os.path.join(image_path_og,img)
    label_path = os.path.join(label_path_og,lab)
    m = img.split('leftImg8bit')
    l = lab.split('gtFine_labelIds')
    if m==l:
      x.append(image_path)
      y.append(label_path)
    else:
      print('[ERR]Data Mismatch')
  if cat == 'train':
    return x[2963:],y[2963:]
  else:
    return x,y

def read_image(path):
  x = cv.imread(path)
  x = cv.cvtColor(x,cv.COLOR_BGR2RGB)
  return x

def read_mask(path):
    x = cv.imread(path,cv.IMREAD_UNCHANGED)
    x = np.expand_dims(x,2)
    # x = (np.arange(x.max()+1) == x[...,None]).astype(np.int8)
    return x

def preprocess(x, y):
  def f(x, y):
    x = x.decode()
    y = y.decode()

    x = read_image(x)
    y = read_mask(y)

    return x, y

  images, masks = tf.numpy_function(f, [x, y], [tf.uint8, tf.uint8])
  
  images.set_shape([512, 1024, 3])
  masks.set_shape([512, 1024, 1])

  return images, masks

def tf_dataset(x, y, batch=6):
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.map(preprocess)
  dataset = dataset.batch(batch)
  dataset = dataset.prefetch(2)
  return dataset

val_images, val_masks = load_data('val')
val_dataset = tf_dataset(val_images, val_masks)

train_images, train_masks = load_data('train')
train_dataset = tf_dataset(train_images, train_masks)

In [ ]:
filepath = os.path.join('drive','MyDrive','CITY','fast_scnn_weights_3.hdf5')
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='accuracy',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    save_freq=45,
)
H = fast_scnn.fit(train_dataset, epochs = 1, validation_data = (val_dataset),callbacks=[checkpoint])
fast_scnn.save_weights(os.path.join('drive','MyDrive','CITY','fast_scnn_weight_last.h5'))
model_path = os.path.join('drive','MyDrive','CITY','fast_scnn.h5')
fast_scnn.save(model_path)

KeyboardInterrupt: ignored

Data Augmentation

In [ ]:
transform = A.Compose([
    A.RandomCrop(height=716,width=1433,p=1.0),
    A.HorizontalFlip(p=.65),
#     A.RandomBrightness(limit=(0.1,0.15),p=.8),
#     A.RandomContrast(limit=(0.3,0.5),p=.8),
#     A.transforms.HueSaturationValue(hue_shift_limit=0,sat_shift_limit=(5,15),val_shift_limit=0,p=.6),
    A.Resize(height = 1024, width = 2048) 
])

# x = np.empty((3475,1024,2048,3))
y = []

for cat in ['train','val']: #
    
    img_path = os.path.join(image_path,cat)
    lab_path = os.path.join(label_path,cat)
    # img_order_path = os.path.join(file_order_path,'img_{}_file_order.npy'.format(cat))
    lab_order_path = os.path.join(file_order_path,'label_{}_file_order.npy'.format(cat))
    # img_order = np.load(img_order_path)
    lab_order = np.load(lab_order_path)
    
    # if cat == 'train':
    #   img_order = img_order[:80]
    #   lab_order = lab_order[:80]
    # elif cat == 'val':
    #   img_order = img_order[:20]
    #   lab_order = lab_order[:20]
    # for idx,img,lab in tqdm(enumerate(zip(img_order,lab_order)),total=len(img_order)):
    for lab in tqdm(lab_order,total = len(lab_order)):
        # imag_path = os.path.join(img_path,img)
        labe_path = os.path.join(lab_path,lab)
        # image = cv.imread(imag_path)
        # image = cv.cvtColor(image,cv.COLOR_BGR2RGB)
        label = cv.imread(labe_path,cv.IMREAD_UNCHANGED)
        # x[idx] = image
        y.append(label)

Model: "Fast_SCNN"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 512, 1024,   0           []                               
                                3)]                                                               
                                                                                                  
 conv2d_26 (Conv2D)             (None, 256, 512, 32  896         ['input_layer[0][0]']            
                                )                                                                 
                                                                                                  
 batch_normalization_36 (BatchN  (None, 256, 512, 32  128        ['conv2d_26[0][0]']              
 ormalization)                  )                                                         

### Data Pipeline

In [ ]:
# total_aug = 2
# all_images = []
# all_labels = []
# aug_images = []
# aug_labels = []

# for cat in ['train','val']: #
    
#     img_path = os.path.join(image_path,cat)
#     lab_path = os.path.join(label_path,cat)
#     img_order_path = os.path.join(file_order_path,'img_{}_file_order.npy'.format(cat))
#     lab_order_path = os.path.join(file_order_path,'label_{}_file_order.npy'.format(cat))
#     img_order = np.load(img_order_path)
#     lab_order = np.load(lab_order_path)
#     if cat == 'train':
#       img_order = img_order[:800]
#       lab_order = lab_order[:800]
#     elif cat == 'val':
#       img_order = img_order[:200]
#       lab_order = lab_order[:200]
#     for img,lab in tqdm(zip(img_order,lab_order),total=len(img_order)):
#         imag_path = os.path.join(img_path,img)
#         labe_path = os.path.join(lab_path,lab)
#         image = cv.imread(imag_path)
#         image = cv.cvtColor(image,cv.COLOR_BGR2RGB)
#         label = cv.imread(labe_path,cv.IMREAD_UNCHANGED)
#         all_images.append(image)
#         all_labels.append(label)
#         # for i in range(total_aug):
#         #     transformed = transform(image=image,mask=label)
#         #     new_image = transformed['image']
#         #     new_label = transformed['mask']
#         #     aug_images.append(new_image)
#         #     aug_labels.append(new_label)

In [ ]:
fast_scnn.summary()

Model: "Fast_SCNN"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 512, 1024,   0           []                               
                                3)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 256, 512, 32  896         ['input_layer[0][0]']            
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 512, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                         

In [ ]:
image_path = os.path.join('drive','MyDrive','cityscape','Resized','images')
label_path = os.path.join('drive','MyDrive','cityscape','Resized','labels_mod')
file_order_path = os.path.join('drive','MyDrive','cityscape','File_order')

x = []
y = []

for cat in ['train','val']: 

    img_order_path = os.path.join(file_order_path,'img_{}_file_order.npy'.format(cat))
    lab_order_path = os.path.join(file_order_path,'label_{}_file_order.npy'.format(cat))
    img_order = np.load(img_order_path)
    lab_order = np.load(lab_order_path)
    
    for img,lab in tqdm(zip(img_order,lab_order),total = len(img_order)):
        imag_path = os.path.join(image_path,img)
        labe_path = os.path.join(label_path,lab)
        image = cv.imread(imag_path)
        image = cv.cvtColor(image,cv.COLOR_BGR2RGB)
        label = cv.imread(labe_path,cv.IMREAD_UNCHANGED)
        label = (np.arange(label.max()+1) == label[...,None]).astype(np.int8)
        x.append(image)
        y.append(label)
# x = np.array(x)
# y = np.array(y,dtype = np.int8)

100%|██████████| 500/500 [00:23<00:00, 21.68it/s]


In [ ]:
fast_scnn

In [ ]:
for data in train_dataset:
  # print(x[0].shape)
  # break
  print(data)
  break

InvalidArgumentError: ignored